<a href="https://colab.research.google.com/github/Devanshi-cloud/AR/blob/main/Resteraunts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install julep -U --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.7/381.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.4/118.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/739.1 kB 17.2 MB/s eta 0:00:00


In [2]:
import uuid

AGENT_UUID = uuid.uuid4()
TASK_UUID = uuid.uuid4()

In [7]:
from julep import Client
from google.colab import userdata

# Get the secret from Colab's secrets storage
JULEP_API_KEY = userdata.get('JULEP_API_KEY')

# Create the client
client = Client(api_key=JULEP_API_KEY, environment="production")


In [36]:
# Defining the agent
name = "Jacob"
about = "A travel assistant that checks today's weather and suggests indoor or outdoor dining."

# Create the agent
agent = client.agents.create_or_update(
    agent_id=AGENT_UUID,
    name=name,
    about=about,
    model="gpt-4o",
)

In [11]:
!pip install tavily-python

In [21]:
import requests
from google.colab import userdata

tavily_api_key = userdata.get('tavily_api_key')
headers = {"Authorization": f"Bearer {tavily_api_key}"}

def search_tavily(query):
    response = requests.post(
        "https://api.tavily.com/search",
        headers=headers,
        json={"query": query, "search_depth": "basic"}
    )
    result = response.json()
    print(f"Query: {query}")
    print("Tavily raw response:", result)
    return result



In [24]:
def get_iconic_dishes(city):
    query = f"3 iconic dishes of {city}"
    response = search_tavily(query)

    # Extract the 'results' list safely
    results = response.get('results', [])

    # Return the first 3 dishes or less if fewer results
    return results[:3]

# Example usage:
city = "Delhi"
dishes = get_iconic_dishes(city)
print(dishes)  # Should print something like ['Dish1', 'Dish2', 'Dish3']


Query: 3 iconic dishes of Delhi
Tavily raw response: {'query': '3 iconic dishes of Delhi', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Discovering Delhi: Top 10 Famous And Iconic Dishes Of The ... - Slurrp', 'url': 'https://www.slurrp.com/article/discovering-delhi-top-10-famous-and-iconic-dishes-of-the-national-capital-1702045207141', 'content': "Parathas in Delhi are so iconic that there is a whole street named after the dish in the Old Delhi area. If you haven't been to Parathe Wali Gali in Delhi and savoured the myriads of stuffed parathas and kuchas paired with raita, dahi and pickles, then your trip to the national capital is truly incomplete.", 'score': 0.834852, 'raw_content': None}, {'title': 'Top 20 Famous Dishes of Delhi - Crazy Masala Food', 'url': 'https://www.crazymasalafood.com/top-20-famous-dishes-delhi/', 'content': 'This is another famous dish of Delhi, made for non- vegetarians. A must try dish for them. Vegetable salad or chopped

In [41]:
import yaml
from google.colab import userdata

openweathermap_api_key = userdata.get("openweathermap_api_key")

# Defining the task
# This task expects pre-fetched weather, dishes, and restaurant data per city

task_def = yaml.safe_load(f'''
# yaml-language-server: $schema=https://raw.githubusercontent.com/julep-ai/julep/refs/heads/dev/schemas/create_task_request.json
name: One-Day Foodie Tour Planner
description: Creates a weather-based foodie tour with iconic dishes and top-rated restaurants.

input_schema:
  type: object
  properties:
    cities:
      type: array
      items:
        type: object
        properties:
          city:
            type: string
          weather:
            type: string
          dining_type:
            type: string
          dishes:
            type: array
            items:
              type: string
          restaurants:
            type: array
            items:
              type: string
      description: List of city data with weather, dining type, dishes, and restaurants.

tools: []

main:
# Step: Create the foodie itinerary per city
- over: $ steps[0].input.cities
  map:
    prompt:
    - role: system
      content: >-
        You are a gourmet trip planner. Create a one-day foodie itinerary in the given city.
        Use weather to recommend indoor or outdoor dining and include breakfast, lunch, and dinner
        with the iconic dishes and top restaurant names.
    - role: user
      content: >-
        City: {{_.city}}
        Weather: {{_.weather}}
        Dining Type: {{_.dining_type}}
        Dishes: {{_.dishes}}
        Restaurants: {{_.restaurants}}
    unwrap: true

# Step: Final Compilation
- evaluate:
    foodie_tours: |-
      $ '\\n\\n======\\n\\n'.join(tour for tour in _)
''')


In [42]:
# creating the task object
task = client.tasks.create_or_update(
    task_id=TASK_UUID,
    agent_id=AGENT_UUID,
    **task_def
)

In [44]:
cities = ["Delhi", "Mumbai"]

def get_city_data(city):
    weather = "Sunny"  # or use your OpenWeather API result here
    dishes = [res['title'] for res in search_tavily(f"3 iconic dishes of {city}")['results'][:3]]
    restaurants = [res['title'] for res in search_tavily(f"top-rated restaurants in {city} serving {', '.join(dishes)}")['results'][:3]]
    dining_type = "outdoor" if "Sunny" in weather else "indoor"

    return {
        "city": city,
        "weather": weather,
        "dining_type": dining_type,
        "dishes": dishes,
        "restaurants": restaurants
    }

city_data = [get_city_data(city) for city in cities]


Query: 3 iconic dishes of Delhi
Tavily raw response: {'query': '3 iconic dishes of Delhi', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Discovering Delhi: Top 10 Famous And Iconic Dishes Of The ... - Slurrp', 'url': 'https://www.slurrp.com/article/discovering-delhi-top-10-famous-and-iconic-dishes-of-the-national-capital-1702045207141', 'content': "Parathas in Delhi are so iconic that there is a whole street named after the dish in the Old Delhi area. If you haven't been to Parathe Wali Gali in Delhi and savoured the myriads of stuffed parathas and kuchas paired with raita, dahi and pickles, then your trip to the national capital is truly incomplete.", 'score': 0.834852, 'raw_content': None}, {'title': 'Top 20 Famous Dishes of Delhi - Crazy Masala Food', 'url': 'https://www.crazymasalafood.com/top-20-famous-dishes-delhi/', 'content': 'This is another famous dish of Delhi, made for non- vegetarians. A must try dish for them. Vegetable salad or chopped

In [45]:
execution = client.executions.create(
    task_id=task.id,
    input={
        "cities": city_data
    }
)


In [47]:
execution = client.executions.create(
    task_id=task.id,
    input={
        "cities": city_data
    }
)

print("Started an execution. Execution ID:", execution.id)


Started an execution. Execution ID: 068417fc-935f-75cc-8000-2b9dae1c9bf4


In [48]:
import time

max_retries = 30  # e.g. max wait time 30*5=150 seconds
retries = 0

while retries < max_retries:
    execution = client.executions.get(execution.id)
    print("Execution status:", execution.status)
    if execution.status == "succeeded":
        break
    elif execution.status in ["failed", "error"]:
        print("Execution failed with status:", execution.status)
        break
    time.sleep(5)
    retries += 1
else:
    print("Timeout reached waiting for execution.")

if execution.status == "succeeded":
    if 'final_plan' in execution.output:
        print(execution.output['final_plan'])
    else:
        print(execution.output)

Execution status: succeeded
{'foodie_tours': '**Delhi Gourmet Outdoor Foodie Itinerary**\n\n**Weather in Delhi: Sunny**  \nEnjoy the beautiful sunny weather with alfresco dining options that let you bask in the vibrant Delhi atmosphere while savoring some of the city\'s most iconic dishes. \n\n**Breakfast:**\n\n- **Restaurant**: **Indian Accent at The Lodhi** — Start your day at this elegant restaurant known for its inventive presentation of traditional Indian cuisine. Although primarily known for its fine dining, it offers a delightful breakfast on their outdoor terrace.\n  \n- **Dish**: Begin with a refreshing **Parsi Eggs** — a classic spicy scrambled eggs dish flavored with green chilies and cilantro, known for its burst of flavors.\n\n**Mid-Morning Activity:**\n\n- **Visit the India Gate and Enjoy the surrounding gardens**. It’s a perfect opportunity to walk off your breakfast while taking in one of Delhi\'s historic landmarks.\n\n**Lunch:**\n\n- **Restaurant**: **Dilli Haat** — T

In [49]:
# Lists all the task steps that have been executed up to this point in time
transitions = client.executions.transitions.list(execution_id=execution.id).items

# Transitions are retrieved in reverse chronological order
for transition in reversed(transitions):
    print("Transition type: ", transition.type)
    print("Transition output: ", transition.output)
    print("-"*50)

Transition type:  init
Transition output:  {'cities': [{'city': 'Delhi', 'dishes': ['Discovering Delhi: Top 10 Famous And Iconic Dishes Of The ... - Slurrp', 'Top 20 Famous Dishes of Delhi - Crazy Masala Food', 'The Most Popular Dishes in Delhi, India - Culture Trip'], 'weather': 'Sunny', 'dining_type': 'outdoor', 'restaurants': ['23 Most Famous Restaurants in Delhi Not to Miss 2025', '10 Must-Try Restaurants in Delhi in 2025 | Best Places to Eat in the ...', 'The 38 Best Restaurants in New Delhi, India - Eater']}, {'city': 'Mumbai', 'dishes': ['Famous food in Mumbai (and where to eat them) - Condé Nast Traveller India', '76 Best Mumbaikar Foods - TasteAtlas', '10 Local Dishes in Mumbai You Need to Bite Into - NDTV Food'], 'weather': 'Sunny', 'dining_type': 'outdoor', 'restaurants': ['THE 10 BEST Restaurants in Mumbai (Updated June 2025) - Tripadvisor', 'These are the 12 best restaurants in Mumbai right now', 'The 38 Best Restaurants in Mumbai']}]}
-------------------------------------